# Introduction and TCGA background

Liver cancer has the second highest worldwide cancer mortality rate and has limited therapeutic options.

The most common form of liver cancer is hepatocellular carcinoma (HCC), making up 80% of liver cancer cases in the US. HCC is the 9th leading cause of cancer-related deaths in the US, and third worldwide.[1](https://www.cancer.gov/about-nci/organization/ccg/research/structural-genomics/tcga/studied-cancers/liver)


A consortium known as The Cancer Genome Atlas (TCGA) network has made a plethora of molecular information on various cancers, including HCC publicly available. This data include genetic, molecular, diagnostic, demographic, and clinical information from over 350 patients with HCC. 

While surgery can be an effective method for treating early-stage liver cancer, there are few options for more advanced and metastatic tumors. Sorafenib and regorafinib are the only approved treatments for managing advanced HCC, and many others potential therapeutics have failed in Phase III clinical trials. [Previous studies](https://www.cell.com/cell/fulltext/S0092-8674(17)30639-6) studying genomic alterations in samples have found frequently-mutated genes LZTR1, EEF1A1, SF3B1, and SMARCA4.

In addition to DNA alterations however, different biological and tumor microenvrionment factors can influence disease progression. A transcriptomic survey of tissues at various stages of disease progression could help elucidate some of the underlying pathways contributing to tumorigenesis.

## RNA Seq

In this post, we'll be exploring a liver cancer RNA-seq dataset from TCGA. TCGA represents one of the most comprehensive publically-available datasets for cancer research. This tutorial will walk through several steps in how to obtain, process, explore and model gene expression data as it relates to cancer genomics.

In biology, copies of our DNA are used as a template for creating different biological products and proterins. This intermediate copy is known as RNA. Unlike DNA, which is relatively static, RNA can change dynamically depending on environmental factors and stimuli. Technological and commercial advances in next-generation sequencing provided cost-effective methods for conducting RNA-sequencing (RNA-seq) in experiments and a plethora of data. 

Traditionally, RNA is extracted from a sample, and transcribed to a more stable DNA copy(known as complementary DNA because it's the complement to the RNA sample). Sequencing works by attaching one side of each RNA molecule to a surface and repeatedly cycling through a process of attaching colored probes that bind to the RNA, imaging the probe, and washing off the probe. Rinse and repeat.

Following this process we now have tons of strings of T's,C's,G's, and A's that has been barcoded on either side of the sequence. Since each barcode attaches to a known and unique sequence of a gene, we can match these sequences to a reference genome.

Once the sequences have been mapped to their corresponding genes, we now have a quantification of gene expression for our samples. Depending on the question we are looking to answer, comparisons of gene expression between experimental conditions can be used to understand whether particular genetic components and molecular pathways influence that treatment. For instance by performing RNA-seq on normal and tumor liver tissue can show granular differences in actively expressed (or suppressed) genes and provide insight into the role of certain gentic components in influencing tumor progression.

## TODO

Intro to the data


Preparing our data


## Accessing the data

**We'll be focusing on using RNA-seq data from LIHC combined with clinical attributes to identify biomarkers for disease progression.**

TCGA provides raw RNA-seq reads and other clinical data [here](https://portal.gdc.cancer.gov/projects/TCGA-LIHC) and in the R package _[RTCGA](http://rtcga.github.io/RTCGA/)_. This data is already outputted in the accompanying [GitHub](https://github.com/nyhais/hacknights/tree/master/tcga-series/post) as well.

We will be investigating the [Hepatocellular carcinoma dataset(https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5680778/). The TCGA RNASeq data is illumina hiseq Level 3 RSEM normalized expression data. You can read about thec RSEM method [here](https://academic.oup.com/bioinformatics/article/26/4/493/243395).

Essentially this is the raw counts of reads that aligned to the gene transcript, though it's only a guess by the program. Since it's a guess, the values are rational numbers. To simplify things, we'll round the values to the next whole integer. 

Before we start any kind of modeling, it's a good idea to observe some high levels patterns within our data to make sure that with all the steps in our process, our data looks accurate.

RNA-seq datasets provide a unique challenge in that they tend to be very wide (~20k genes for a human genome) and short (sequencing information on clinical samples can be expensive or unavailable and certain conditions may be very rare).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_dir="data/"
response_name="patient.race"
rnaseq_file=data_dir+"lihc_rnaseq.csv.gz"
clinical_file=data_dir+"lihc_clinical.csv.gz"

First let's load the data:

In [ ]:
rnaseq = (pd.
          read_csv(rnaseq_file,compression="gzip").
          set_index('bcr_patient_barcode').
          applymap(lambda x : int(np.ceil(x)))
         )
display(rnaseq.shape)
display(rnaseq.head())

We can clean up the column titles by extracting the common gene symbols from each.

In [ ]:
gene_name_logical = [len(x[0])>1 for x in rnaseq.columns.str.split('|')]
sub = rnaseq.loc[:,gene_name_logical]
sub.columns = [x[0] for x in sub.columns.str.split('|')]
rnaseq_sub = sub.copy()
rnaseq_sub.head()

We'll also load the clinical data from TCGA

In [ ]:
clinical = pd.read_csv(clinical_file,compression="gzip").set_index('patient.bcr_patient_barcode')
display(clinical.shape)
display(clinical.head())

In [ ]:
Questions:
What is the range of values for a given gene?
What is the distribution of values for a given gene?
Are there higher than average or lower than average expression of genes?
Which genes are differentially expressed? Are they positively or negatively expressed compared to your control?
What do these genes do? Which pathways are they involved in?
Are there related clinical phenotypes which might show similar differences in expression?
How many unique values are there for a given clinical attribute?
How can we define an appropriate response variable for supervised learning?
What clinical attributes can be used to identify biomarkers for disease progression?
Background and problem setting - Matt
Liver Cancer
TCGA
RNA-seq and data introduction
Setting up Spell AI platform - Nick
EDA - Matt/Nick
